In [ ]:
mkdir drive/ && mkdir content/

In [ ]:
cp -r '/kaggle/input/cityscapes/gtFine' '/kaggle/working/content/' && cp -r '/kaggle/input/cityscapes/leftImg8bit' '/kaggle/working/content/'

In [ ]:
# imports
from os.path import join, isdir
from os import listdir, rmdir
from shutil import move, rmtree, make_archive

import os
import cv2
import glob
import pickle
import numpy as np
import tensorflow as tf

from PIL import Image
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Input, Dropout, Activation, Flatten, BatchNormalization, ReLU, LeakyReLU, concatenate
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, AveragePooling2D, GlobalAveragePooling2D, Add

# for checkpoints storage
#from google.colab import drive
#drive.mount('/gdrive')
#drive_root = '/gdrive/My Drive/AIMS DTU/2024 - Summer Projects/Semantic Segmentation/'
drive_root = '/kaggle/working/drive'

COLAB_DIR = '/kaggle/working/content/'
GT_DIR = COLAB_DIR + 'gtFine/'
IMG_DIR = COLAB_DIR + 'leftImg8bit/'

In [ ]:
# collapse child directories
for parent in listdir(GT_DIR):
    parent_dir = GT_DIR + parent
    for child in listdir(parent_dir):
        if isdir(join(parent_dir, child)):
            keep = glob.glob(join(parent_dir, child) + '/*_gtFine_color.png')
            keep = [f.split('/')[-1] for f in keep]
            for filename in list(set(listdir(join(parent_dir, child))) & set(keep)):
                move(join(parent_dir, child, filename), join(parent_dir, filename))
            rmtree(join(parent_dir, child))

for parent in listdir(IMG_DIR):
    parent_dir = IMG_DIR + parent
    for child in listdir(parent_dir):
        if isdir(join(parent_dir, child)):
            for filename in listdir(join(parent_dir, child)):
                move(join(parent_dir, child, filename), join(parent_dir, filename))
            rmtree(join(parent_dir, child))

In [ ]:
# normalize image pixels (z-score to impliment)
IMG_SIZE = 256
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE  # auto tunes the pipeline's performance

def load_and_preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img /= 256.0
    return img


def get_image_paths(dir):
#    return sorted([os.path.join(dir, path) for path in os.listdir(dir)])
    return sorted([dir + path for path in listdir(dir)])

In [ ]:
# create tf.Dataset objects
gt_train_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(GT_DIR + 'train/'))
gt_val_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(GT_DIR + 'val/'))
gt_test_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(GT_DIR + 'test/'))

gt_train_ds = gt_train_ds.map(load_and_preprocess_image)
gt_val_ds = gt_val_ds.map(load_and_preprocess_image)
gt_test_ds = gt_test_ds.map(load_and_preprocess_image)

im_train_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR + 'train/'))
im_val_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR + 'val/'))
im_test_ds = tf.data.Dataset.from_tensor_slices(get_image_paths(IMG_DIR + 'test/'))

im_train_ds = im_train_ds.map(load_and_preprocess_image)
im_val_ds = im_val_ds.map(load_and_preprocess_image)
im_test_ds = im_test_ds.map(load_and_preprocess_image)

In [ ]:
# visualize the data
def visualize_images(img, gt, pred):
    if pred is not None:
        fig, axes = plt.subplots(1, 3, figsize=(12, 8))
    else:
        fig, axes = plt.subplots(1, 2, figsize=(8, 8))

    axes[0].imshow(img)
    axes[0].set_title('Actual Image')

    axes[1].imshow(gt)
    axes[1].set_title('Masked Image')
    
    if pred is not None:
        axes[2].imshow(pred)
        axes[2].set_title('Predicted Image')

In [ ]:
# visualizing image with corresponding mask
for img, gt in list(zip(im_train_ds.take(2), gt_train_ds.take(2))):
    visualize_images(img, gt, None)

In [ ]:
# resizing class
class ResizeLayer(tf.keras.layers.Layer):
    def __init__(self, size, **kwargs):
        super(ResizeLayer, self).__init__(**kwargs)
        self.size = size
        
    def call(self, inputs):
        return tf.image.resize(inputs, self.size)

In [ ]:
# pspnet architecture implimentation
def conv_block(X, filters, block):
    # resiudal block with dilated convolutions
    # add skip connection at last after doing convoluion

    b = 'block_' + str(block) + '_'
    f1, f2, f3 = filters
    X_skip = X

    # block_a
    X = Conv2D(filters=f1, kernel_size=(1, 1), dilation_rate=(1, 1),
               padding='same', kernel_initializer='he_normal', name=b + 'a')(X)
    X = BatchNormalization(name=b + 'batch_norm_a')(X)
    X = LeakyReLU(alpha=0.2, name=b + 'leakyrelu_a')(X)
    # block_b
    X = Conv2D(filters=f2, kernel_size=(3, 3), dilation_rate=(2, 2),
               padding='same', kernel_initializer='he_normal', name=b + 'b')(X)
    X = BatchNormalization(name=b + 'batch_norm_b')(X)
    X = LeakyReLU(alpha=0.2, name=b + 'leakyrelu_b')(X)
    # block_c
    X = Conv2D(filters=f3, kernel_size=(1, 1), dilation_rate=(1, 1),
               padding='same', kernel_initializer='he_normal', name=b + 'c')(X)
    X = BatchNormalization(name=b + 'batch_norm_c')(X)
    # skip_conv
    X_skip = Conv2D(filters=f3, kernel_size=(3, 3), padding='same', name=b + 'skip_conv')(X_skip)
    X_skip = BatchNormalization(name=b + 'batch_norm_skip_conv')(X_skip)
    # block_c + skip_conv
    X = Add(name=b + 'add')([X, X_skip])
    X = ReLU(name=b + 'relu')(X)
    return X


def base_feature_maps(input_layer):
    # base covolution module to get input image feature maps

    # block_1
    base = conv_block(input_layer, [16, 16, 32], '1')
    # block_2
    base = conv_block(base, [16, 16, 32], '2')
    return base


def pyramid_feature_maps(input_layer):
    # pyramid pooling module

    base = base_feature_maps(input_layer)
    # red
    red = GlobalAveragePooling2D(name='red_pool')(base)
    red = tf.keras.layers.Reshape((1, 1, 32))(red)
    red = Conv2D(filters=32, kernel_size=(1, 1), name='red_1_by_1')(red)
    red = UpSampling2D(size=128, interpolation='bilinear', name='red_upsampling')(red)
#    red = tf.image.resize(red, [IMG_SIZE, IMG_SIZE])
    red = ResizeLayer(size=(IMG_SIZE, IMG_SIZE))(red)
    
    # yellow
    yellow = AveragePooling2D(pool_size=(2, 2), name='yellow_pool')(base)
    yellow = Conv2D(filters=32, kernel_size=(1, 1), name='yellow_1_by_1')(yellow)
    yellow = UpSampling2D(size=2, interpolation='bilinear', name='yellow_upsampling')(yellow)
#    yellow = tf.image.resize(yellow, [IMG_SIZE, IMG_SIZE])
    yellow = ResizeLayer(size=(IMG_SIZE, IMG_SIZE))(yellow)  
    
    # blue
    blue = AveragePooling2D(pool_size=(4, 4), name='blue_pool')(base)
    blue = Conv2D(filters=32, kernel_size=(1, 1), name='blue_1_by_1')(blue)
    blue = UpSampling2D(size=4, interpolation='bilinear', name='blue_upsampling')(blue)
#    blue = tf.image.resize(blue, [IMG_SIZE, IMG_SIZE])
    blue = ResizeLayer(size=(IMG_SIZE, IMG_SIZE))(blue)
    
    # green
    green = AveragePooling2D(pool_size=(8, 8), name='green_pool')(base)
    green = Conv2D(filters=32, kernel_size=(1, 1), name='green_1_by_1')(green)
    green = UpSampling2D(size=8, interpolation='bilinear', name='green_upsampling')(green)
#    green = tf.image.resize(green, [IMG_SIZE, IMG_SIZE])
    green = ResizeLayer(size=(IMG_SIZE, IMG_SIZE))(green)
    
    
    # base + red + yellow + blue + green
    base = ResizeLayer(size=(IMG_SIZE, IMG_SIZE))(base)
    return tf.keras.layers.concatenate([base, red, yellow, blue, green])


def last_conv_module(input_layer):
    X = pyramid_feature_maps(input_layer)
    X = Conv2D(filters=3, kernel_size=3, padding='same', name='last_conv_3_by_3')(X)
    X = BatchNormalization(name='last_conv_3_by_3_batch_norm')(X)
    X = Activation('sigmoid', name='last_conv_relu')(X)
    return X

In [ ]:
input_shape = list(im_train_ds.take(1))[0].shape
input_layer = tf.keras.Input(shape=input_shape, name='input')
output_layer = last_conv_module(input_layer)

model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.summary()

## Model Training & Evaluation

In [ ]:
train_ds = tf.data.Dataset.zip((im_train_ds, gt_train_ds))
train_ds = train_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

val_ds = tf.data.Dataset.zip((im_val_ds, gt_val_ds))
val_ds = val_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

test_ds = tf.data.Dataset.zip((im_test_ds, gt_test_ds))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

In [ ]:
checkpoint_path = drive_root+'pspnet/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
es_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model.compile(optimizer='adam', loss='mse', metrics='accuracy')
history = model.fit(train_ds, validation_data=val_ds, epochs=5, 
                    callbacks=[cp_callback, es_callback])
model.save(drive_root + 'pspnet_trained.h5')

In [ ]:
# train and val accuracy and loss vs epochs
def plot(history):
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(1,len(acc)+1)

  plt.title('Training and validation accuracy')
  plt.plot(epochs, acc, color='blue', label='Train')
  plt.plot(epochs, val_acc, color='orange', label='Val')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.legend()

  _ = plt.figure()
  plt.title('Training and validation loss')
  plt.plot(epochs, loss, color='blue', label='Train')
  plt.plot(epochs, val_loss, color='orange', label='Val')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()
  
plot(history)